In [8]:
import yaml
import pymongo
from urllib.parse import quote_plus as quote
import numpy as np

collection_name = 'augmented_dataset' #['unique_dataset', 'augmented_dataset', 'initial_dataset']

In [9]:
# Подключимся к нашей коллекции

with open('../env/credsw.yaml', 'r') as file:
    creds_dict = yaml.safe_load(file)
    
url = 'mongodb://{user}:{pw}@{hosts}/?{rs}&authSource={auth_src}&{am}&tls=true&tlsCAFile={cert_file}'.format(
    user=creds_dict['username'],
    pw=quote(creds_dict['password']),
    hosts=creds_dict['host'],
    rs='replicaSet=rs01',
    auth_src=creds_dict['database'],
    am='authMechanism=DEFAULT',
    cert_file='../env/root.crt'
    )

dbs = pymongo.MongoClient(url)

db = dbs[creds_dict['database']]

collection = db[collection_name]

In [10]:
# Выгрузим данные из коллекции
pipeline = [{ "$unwind" : "$boxes" },
            {"$project":{
                "size": "$boxes.size",
                "stacking": "$boxes.stacking",
                "turnover": "$boxes.turnover",
                "loading_size": "$loading_size",
                "density_percent": "$density_percent"
                }},
            {"$group": {"_id": "$_id",
                        "loading_size": {"$first":"$loading_size"},
                        "density_percent": {"$first":"$density_percent"},
                        "boxes": {
                            "$push":  {
                                "size": "$size",
                                "stacking": "$stacking",
                                "turnover": "$turnover"                         
                            }
                            
                        }
            }
            }
            ]
result = collection.aggregate(pipeline, allowDiskUse=True)

In [11]:
# Подготовим train и test datasets
test_size = 1000

dataset = [
    (item['density_percent'],
    [[item['loading_size']['width'], item['loading_size']['height'], item['loading_size']['length'], False, False]] + \
    [ [box['size']['width'], box['size']['height'], box['size']['length'], box['stacking'], box['turnover']] for box in item['boxes']])
    for item in result
]

y_train, y_test = [item[0] for item in dataset[0:-test_size]], [item[0] for item in dataset[-test_size : ]]
X_train, X_test = [item[1] for item in dataset[0:-test_size]], [item[1] for item in dataset[-test_size:]]